In [1]:
#importing libraries and data
import pandas as pd
import numpy as np
df = pd.read_csv('Data_for_UCI_named.csv')
df.head()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stab,stabf
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,0.055347,unstable
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,-0.005957,stable
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,0.003471,unstable
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718,0.028871,unstable
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923,0.049860,unstable


In [2]:
df[ 'stabf' ].value_counts()

unstable    6380
stable      3620
Name: stabf, dtype: int64

In [3]:
df.isna().sum()
#for simplicity, we will drop the rows with missing values.
df = df.dropna()
df.head()
df[ 'stabf' ].value_counts()

unstable    6380
stable      3620
Name: stabf, dtype: int64

In [4]:
data_df = df.drop(columns=['stab'])
X = data_df.drop(columns= 'stabf' )
y = data_df[ 'stabf' ]

In [5]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size= 0.2 , random_state= 1 )
#y_train.value_counts()
#print(y_train)

In [6]:
#import imblearn
#from imblearn.over_sampling import SMOTE
#smote = SMOTE(random_state= 1 )
#x_balanced, y_balanced = smote.fit_sample(x_train, y_train)
#print(x_balanced)

x_balanced = x_train
y_balanced = y_train 

In [7]:
#Normalising train df
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler() 

normalised_train_df = scaler.fit_transform(x_balanced)

normalised_train_df = pd.DataFrame(normalised_train_df, columns=(x_balanced).columns)

print(normalised_train_df)

          tau1      tau2      tau3      tau4        p1        p2        p3  \
0     0.367327 -0.986042  0.650447  1.547527 -0.291490  0.061535  1.293862   
1    -0.064659  0.089437  1.035079 -1.641494  0.619865 -0.067235 -1.502925   
2    -1.467850  1.298418 -0.502536  1.166046 -0.180521  0.490603  0.682560   
3     0.820081  0.529920  1.299657 -1.141975 -0.812854 -0.763632  1.521579   
4     0.665424 -1.425627  0.312300  0.919137 -1.614296  0.760315  1.422019   
...        ...       ...       ...       ...       ...       ...       ...   
7995  1.551314  0.007408 -1.177640  1.016898 -0.397177  0.759820 -0.636951   
7996  1.015925 -0.223483 -1.489381 -1.479078  0.451468 -0.731994  0.990355   
7997  0.657609 -0.722756 -1.405888 -0.274301 -0.012584  1.438694 -0.364266   
7998 -0.059316 -1.260532 -1.010471 -0.877808 -0.779769  0.828824  0.516923   
7999 -1.473214  0.638438  0.250122 -0.996484  1.950944 -1.163800 -0.732842   

            p4        g1        g2        g3        g4  
0    -

In [8]:
#Normalising test df
normalised_test_df = scaler.transform (x_test)
normalised_test_df = pd.DataFrame(normalised_test_df, columns=(x_balanced).columns)

In [9]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score, accuracy_score, precision_score, f1_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.ensemble import ExtraTreesClassifier
from pprint import pprint
from sklearn.ensemble import RandomForestRegressor


In [10]:
#xgboost
# fit model no training data using Xgb
modelxgb = XGBClassifier(random_state=1)
modelxgb.fit(normalised_train_df,y_balanced)

xgb_pred=modelxgb.predict(normalised_test_df)

# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test,xgb_pred))

#Accuracy 
accuracy = accuracy_score(y_true=y_test, y_pred=xgb_pred) 
print( 'Accuracy: {}' .format(round(accuracy* 100 ), 2 )) 

#Precision 
precision = precision_score(y_true=y_test, y_pred=xgb_pred, pos_label= 'unstable' )
print( 'Precision: {}' .format(round(precision* 100 ), 2 ))

#Recall
recall = recall_score(y_true=y_test, y_pred=xgb_pred, pos_label= 'unstable')
print( 'Recall: {}' .format(round(recall* 100 ), 2 )) 

#F1-Score
f1 = f1_score(y_true=y_test, y_pred=xgb_pred, pos_label= 'unstable' )
print( 'F1: {}' .format(round(f1* 100 ), 2 ))




Accuracy: 0.9455
Accuracy: 95.0
Precision: 95.0
Recall: 97.0
F1: 96.0


In [11]:
# build the lightgbm model
import lightgbm as lgb

# fit model no training data using lgb
model_lgb = lgb.LGBMClassifier(random_state=1)
model_lgb.fit(normalised_train_df,y_balanced)

lgb_pred=model_lgb.predict(normalised_test_df)

# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test,lgb_pred))

#Accuracy 
accuracy = accuracy_score(y_true=y_test, y_pred=lgb_pred) 
print( 'Accuracy: {}' .format(round(accuracy* 100 ), 2 )) 

#Precision 
precision = precision_score(y_true=y_test, y_pred=lgb_pred, pos_label= 'unstable' )
print( 'Precision: {}' .format(round(precision* 100 ), 2 )) 

#Recall
recall = recall_score(y_true=y_test, y_pred=lgb_pred, pos_label= 'unstable')
print( 'Recall: {}' .format(round(recall* 100 ), 2 )) 

#F1-Score
f1 = f1_score(y_true=y_test, y_pred=lgb_pred, pos_label= 'unstable' )
print( 'F1: {}' .format(round(f1* 100 ), 2 )) 



Accuracy: 0.9375
Accuracy: 94.0
Precision: 94.0
Recall: 96.0
F1: 95.0


In [12]:
#Import Random Forest Model
from sklearn.ensemble import RandomForestClassifier

#Create a Gaussian Classifier
model_RF=RandomForestClassifier( random_state=1)

#Train the model using the training sets y_pred=clf.predict(X_test)
model_RF.fit(normalised_train_df, y_balanced)

RF_pred=model_RF.predict(normalised_test_df)

#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, RF_pred))

#Accuracy 
accuracy = accuracy_score(y_true=y_test, y_pred=RF_pred) 
print( 'Accuracy: {}' .format(round(accuracy* 100 ), 2 )) 

#Precision 
precision = precision_score(y_true=y_test, y_pred=RF_pred, pos_label= 'unstable' )
print( 'Precision: {}' .format(round(precision* 100), 2 ))

#Recall
recall = recall_score(y_true=y_test, y_pred=RF_pred, pos_label= 'unstable')
print( 'Recall: {}' .format(round(recall* 100 ), 2 )) 

#F1-Score
f1 = f1_score(y_true=y_test, y_pred=RF_pred, pos_label= 'unstable' )
print( 'F1: {}' .format(round(f1* 100 ), 2 ))

model_RF.predict([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

Accuracy: 0.929
Accuracy: 93.0
Precision: 93.0
Recall: 96.0
F1: 95.0


array(['unstable'], dtype=object)

In [13]:
#Random Hyperparameter Grid
#To use RandomizedSearchCV, we first need to create a parameter grid to sample from during fitting:

from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [50, 100, 300, 500, 1000]

# Number of features to consider at every split
max_features = ['auto', 'sqrt', 'log2', None] 

# Maximum number of levels in tree
#max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
#max_depth.append(None)

# Minimum number of samples required to split a node
min_samples_split = [2, 3, 5, 7, 9]

# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4, 6, 8]

# Method of selecting samples for training each tree
#bootstrap = [True, False]

# Create the random grid
hyperparameter_grid = {'n_estimators': n_estimators,
                       'min_samples_leaf': min_samples_leaf,
                       'min_samples_split': min_samples_split,
                       'max_features': max_features}
pprint(hyperparameter_grid)

{'max_features': ['auto', 'sqrt', 'log2', None],
 'min_samples_leaf': [1, 2, 4, 6, 8],
 'min_samples_split': [2, 3, 5, 7, 9],
 'n_estimators': [50, 100, 300, 500, 1000]}


In [14]:
#Random Search Training
#Now, we instantiate the random search and fit it like any Scikit-Learn model:

# Use the random grid to search for best hyperparameters
# First create the base model to tune
#ranf = RandomForestRegressor()
ranf = ExtraTreesClassifier()

# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator=ranf, param_distributions=hyperparameter_grid,
                              n_iter = 10, scoring='neg_mean_absolute_error', 
                              cv = 5, verbose=1, random_state=1, n_jobs=-1,
                              return_train_score=True)

In [15]:
#encode categorical variable
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
y_balancednew = encoder.fit_transform(y_balanced)
#y_test = encoder.transform(y_test)

In [16]:
# Fit the random search model
rf_random.fit(normalised_train_df, y_balancednew)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   58.6s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  1.1min finished


RandomizedSearchCV(cv=5, estimator=ExtraTreesClassifier(), n_jobs=-1,
                   param_distributions={'max_features': ['auto', 'sqrt', 'log2',
                                                         None],
                                        'min_samples_leaf': [1, 2, 4, 6, 8],
                                        'min_samples_split': [2, 3, 5, 7, 9],
                                        'n_estimators': [50, 100, 300, 500,
                                                         1000]},
                   random_state=1, return_train_score=True,
                   scoring='neg_mean_absolute_error', verbose=1)

In [17]:
#Best rf classifier to use
rf_random.best_params_

{'n_estimators': 1000,
 'min_samples_split': 2,
 'min_samples_leaf': 8,
 'max_features': None}

In [18]:
#Import Random Forest Model HyperParameter
from sklearn.ensemble import RandomForestClassifier

#Create a Gaussian Classifier
new_model_RF=RandomForestClassifier(n_estimators=100, min_samples_split=3, min_samples_leaf = 2, max_features = 'auto', random_state=1)

#Train the model using the training sets y_pred=clf.predict(X_test)
new_model_RF.fit(normalised_train_df, y_balanced)

RF_pred_new = new_model_RF.predict(normalised_test_df)

#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test,RF_pred_new))

#Accuracy 
accuracy = accuracy_score(y_true=y_test, y_pred=RF_pred_new) 
print( 'Accuracy: {}' .format(round(accuracy* 100 ), 2 )) 

#Precision 
precision = precision_score(y_true=y_test, y_pred=RF_pred_new, pos_label= 'stable' )
print( 'Precision: {}' .format(round(precision* 100 ), 2 ))

#Recall
recall = recall_score(y_true=y_test, y_pred=RF_pred_new, pos_label= 'stable')
print( 'Recall: {}' .format(round(recall* 100 ), 2 )) 

#F1-Score
f1 = f1_score(y_true=y_test, y_pred=RF_pred_new, pos_label= 'stable' )
print( 'F1: {}' .format(round(f1* 100 ), 2 ))

Accuracy: 0.9225
Accuracy: 92.0
Precision: 91.0
Recall: 87.0
F1: 89.0


In [19]:
#Import ExtraTreesClassifier
from sklearn.ensemble import ExtraTreesClassifier

#Create a Gaussian Classifier
model_ETC = ExtraTreesClassifier(random_state=1)

#Train the model using the training sets y_pred=clf.predict(X_test)
model_ETC.fit(normalised_train_df, y_balanced)

ETC_pred = new_model_RF.predict(normalised_test_df)

#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test,ETC_pred))

#Accuracy 
accuracy = accuracy_score(y_true=y_test, y_pred=ETC_pred) 
print( 'Accuracy: {}' .format(round(accuracy* 100 ), 2 )) 

#Precision 
precision = precision_score(y_true=y_test, y_pred=ETC_pred, pos_label= 'unstable' )
print( 'Precision: {}' .format(round(precision* 100 ), 2 ))

#Recall
recall = recall_score(y_true=y_test, y_pred=ETC_pred, pos_label= 'unstable')
print( 'Recall: {}' .format(round(recall* 100 ), 2 )) 

#F1-Score
f1 = f1_score(y_true=y_test, y_pred=ETC_pred, pos_label= 'unstable' )
print( 'F1: {}' .format(round(f1* 100 ), 2 ))

Accuracy: 0.9225
Accuracy: 92.0
Precision: 93.0
Recall: 95.0
F1: 94.0
